# 6.12 Final Assignment: Advanced Algorithms Design

### Objective

The objective of this notebook is to create the quantum walk operator for the case of a circle with 4 nodes, and design the quantum walk operator for the case of a line with 16 nodes.


Through this notebook you will understand what is a quantum walk and how to create one.

## Random walks

To understand quantum walks, first we must understand what is a classical random walk.

It is defined as "the process by which randomly-moving objects wander away from where they started" [1]. Mathematically, here is the definition:

Suppose that $X_1$, $X_2$, . . . is a sequence of $I\!R^d$-valued independent and identically distributed random variables. A random walk started at $z \in I\!R^d$ is the sequence $(S_n)_{n≥0}$ where $S_0 = z$ and
$$ S_n = S_{n−1} + X_n, n ≥ 1. $$
Where $X_n$ are referred to as steps of the random walk, $S_n$ is the position at time $n$. [2]


The following GIF shows 7 different random walks in 2 dimension.

 ![Random Walk](https://www.mit.edu/~kardar/teaching/projects/chemotaxis(AndreaSchmidt)/rand_2D.gif)

## Quantum Walk

As seen with other classical algorithms, quantum walks surpass their classical counterpart as they are faster and help to speedup other algorithm [3]. 

In the next cells, you can learn how to create a quantum walk

In [1]:
from classiq import * # import needed libraries

The next functions help for the phase kickback. We use an auxiliary qubit and prepare it in the minus state. Then we apply the diffuzer oracle.

In [2]:
@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)
    

@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)

Now, here we introduce two new variables: size and nodes. Nodes, as the name indicates, is the number of nodes and size the number of qubits we need to do perform the algorithm.

The W operator help us for the phase kickback.

In [3]:
def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0]*nodes # we initialize all the probabilities for all nodes
    # we change the probabilites so that there are 16 different states
    prob[(i+1)% nodes]=0.5 
    prob[(i-1)% nodes]=0.5
    print(f'State={i}, prob vec ={prob}')

    # apply the diffuser after preparing the probability states
    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc 
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size): # 2^size is the capacity of the qubits
      W_iteration(i,vertices,adjacent_vertices)

This is the S operator, first we apply the oracle for the edges and perform a bitwise swap only if the result of the oracle is 1 because that means that there is an edge to another node that the walker can move to.

In [4]:
@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) == 1)


@qfunc 
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))


@qfunc 
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))

# 4 nodes

This is an example for only 4 nodes

In [5]:
nodes = 4 # number of nodes
size = 2 # aka number of qubits used for n nodes: log(4) = 2

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 0.5, 0, 0.5]
State=1, prob vec =[0.5, 0, 0.5, 0]
State=2, prob vec =[0, 0.5, 0, 0.5]
State=3, prob vec =[0.5, 0, 0.5, 0]
Opening: https://platform.classiq.io/circuit/2168192d-7cc5-4e2b-9d5a-4e3ed98587da?version=0.43.3


![nodes 4](nodes_4.jpg)

# 16 nodes

This the code for 16 nodes, in here we only need 2 qubits more to represent all 16 nodes.

In [6]:
nodes = 16 # 
size = 4 # aka number of qubits used for n nodes: log(16) = 4

@qfunc 
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5,

![nodes 16](nodes_16.jpg)

# References

[1] https://www.mit.edu/~kardar/teaching/projects/chemotaxis(AndreaSchmidt)/random.htm

[2] https://www.math.ucla.edu/~biskup/PDFs/PCMI/PCMI-notes-1

[3] https://arxiv.org/abs/1201.4780